In [2]:
!pip install peft
!pip install Faiss-cpu
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

!pip install rank_bm25
!pip install sentencepiece
!pip install sentence-transformers
!pip install pypdf
!pip install "unstructured[all-docs]"
!pip install elasticsearch openai tiktoken langchain
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from langchain.schema import Document
from peft import PeftModel, PeftConfig
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFDirectoryLoader

from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TextStreamer, GenerationConfig

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import Chroma
import torch
import re
import base64
import os
import unicodedata

from google.colab import drive
# 도커로 경로 바꾸기
drive.mount('/content/drive')

# csv load
loader = DirectoryLoader("/content/drive/MyDrive/[FINAL] 텍스트 전처리 통합", loader_cls=TextLoader)
data=loader.load()

loader_graph = DirectoryLoader("/content/drive/MyDrive/[FINAL] 그래프 전처리", loader_cls=TextLoader)
data_graph=loader_graph.load()

# text split
# chunk_overlap - 겹쳐서 나오는 내용 조절
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=30)
data_split_text = text_splitter.split_documents(data)
data_split_graph = text_splitter.split_documents(data_graph)

# embedding
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
db_faiss_text = FAISS.from_documents(data_split_text, embeddings)
db_faiss_graph = FAISS.from_documents(data_split_graph, embeddings)

from rank_bm25 import BM25Okapi

def make_tok(sent):
  return sent.split(" ")

corpus=  [i.page_content for i in data_split_text]
tokenized_corpus = [make_tok(i.page_content) for i in data_split_text]
bm25 = BM25Okapi(tokenized_corpus)

corpus_graph=  [i.page_content for i in data_split_graph]
tokenized_corpus_graph = [make_tok(i.page_content) for i in data_split_graph]
bm25_graph = BM25Okapi(tokenized_corpus_graph)
# search similarity - text
def db_search(query: str, k: int):
    docs = db_faiss_text.similarity_search(query, k)
    return docs

# search similarity - graph
def db_search_graph(query: str, k: int):
    docs = db_faiss_graph.similarity_search(query, k)
    return docs

# Bm25리트리버 - 4개씩 가져오도록 설정, 변경가능
bm25_retriever = BM25Retriever.from_texts(corpus)
bm25_retriever.k = 4
bm25_retriever_graph = BM25Retriever.from_texts(corpus_graph)
bm25_retriever_graph.k = 4

# faiss_retreiver
faiss_retriever = db_faiss_text.as_retriever(search_kwargs={"k": 7})
faiss_retriever_graph = db_faiss_graph.as_retriever(search_kwargs={"k": 7})

# 앙상블 가중치 - Bm25 가중치 0.2, faiss 0.8 가중치
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.2, 0.8])

# 로컬 llm가져오기
def load_local_llm():
  model='davidkim205/komt-mistral-7b-v1'
  # lora학습법 사용
  # peft-parameter efficient fine tunning (일부의 파라미터만 튜닝)
  peft_model_name = 'davidkim205/komt-mistral-7b-v1-lora'
  config = PeftConfig.from_pretrained(peft_model_name)
  # 4비트 양자화를 시켜서 가져옴. 그냥 가져오면 너무 크다
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  config.base_model_name_or_path =model
  model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
  model = PeftModel.from_pretrained(model, peft_model_name)
  return model

model=load_local_llm()
# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(PeftConfig.from_pretrained('davidkim205/komt-mistral-7b-v1-lora').base_model_name_or_path)
streamer = TextStreamer(tokenizer)

# 파일리스트는 가져옴
file_list=os.listdir('/content/drive/MyDrive/[FINAL] 그래프 png 파일')
file_names = file_list

list_of_documents = [
    Document(page_content=name, metadata=dict(mainsource=name.split('_')[0]))
    for name in file_names
]
db_faiss_graghs = FAISS.from_documents(list_of_documents, embeddings)

# gen1 쿼리에 관련된 답변을 1차적으로 생성해 리트리버의 input으로 넣어줌
def gen1(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문 = {x} [/INST]
    """

    q = template
    # q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

# def 그래프 -> 한번 시도해보기. 질문과 가장 관련된 파일을 가져오기
def graphs(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        아래 문서는 그래프에 대한 해석이다.
        문서에 주어진 내용만을 이용해서 답하고 문서에서 근거를 찾을 수 없거나 답변하기 모호하면 정보를 찾을 수 없습니다. 라고 답변해줘.

        문서: {docs_input}
        질문: {x} [/INST]
        output ():
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

# 질문과 관련된 비슷한 질문 생성
def make_similar_query(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        질문과 관련된 2개의 다중 검색 질문를 생성해줘.

        질문: {x} [/INST]
        output (주어진 질문과 유사한 질문 2개):
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

def find_core(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=1,
        top_k=100,
        max_new_tokens=40,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        사실만을 말하는 금융 전문가로서 답해줘.

        질문: {x} [/INST]

    """
    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


# 최종 답변을 생성하는 함수
def gen_final(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {docs_input}
        질문: {x} [/INST]
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

# 질문과 가장 연관있는 그래프 선택
def gengraph(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        주어진 예시중 질문과 가장 관련있는 것을 1개 고르시오.

        문서 제목: {docs_input}
        질문: {x} [/INST]
        output (선택된 문서 1개):
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


def filter_documents_by_query(documents, query):
    # 쿼리에서 키워드 확인
    query_keywords = [unicodedata.normalize('NFD', keyword) for keyword in ["하이브", "퀄컴", "미국 마감시황", "자동차", "롯데",'현대','기아'] if keyword in query]
    # 먼저 pdf의 종류를 고르고, 아래에 해당되는 키워드가 있는지 살펴서 그래프를 가져옴
    # query_keywords2 = [unicodedata.normalize('NFD', keyword) for keyword in ['매출','재고','영업이익','EPS','eps'] if keyword in query]
    # 해당 키워드를 mainsource로 가지는 문서 필터링
    if len(query_keywords)>0:
      filtered_docs = [doc for doc in documents if (any(item in doc.metadata.get('source', '') for item in query_keywords)) or (any(item in doc.page_content for item in query_keywords))]
    else:
      filtered_docs= [doc for doc in documents]
    return filtered_docs

from langchain.document_transformers import (
    LongContextReorder,
)

'''여기서 부터 쿼리 받아서 적용!'''

query=''

while(query!='exit'):
  query=input("원하는 질문을 입력하세요. 종료를 원한다면 'exit'라고 입력하세요 : ")
  if (query=='exit'):
    print('종료합니다.')
    break
  outputs=make_similar_query(query)

  # 1차적인 답변 생성
  core=find_core(query)
  docs = ensemble_retriever.get_relevant_documents(query)
  #docs = [doc.page_content for doc in docs]
  docs2 =ensemble_retriever.get_relevant_documents(core)
  #docs2 = [doc.page_content for doc in docs2]
  docs3 =ensemble_retriever.get_relevant_documents(outputs.split('\n')[1])
  #docs3 = [doc.page_content for doc in docs3]
  docs4 =ensemble_retriever.get_relevant_documents(outputs.split('\n')[2].replace('</s>',''))
  #docs4 = [doc.page_content for doc in docs4]
  docs_all=(docs+docs2+docs3+docs4)
  documents=docs_all

  # 예시 사용
  filtered_documents = filter_documents_by_query(documents, query)

  # 필터링된 문서 정보
  # filtered_documents_info = [(doc.page_content, doc.metadata) for doc in filtered_documents]
  reordering = LongContextReorder()
  reordered_docs = reordering.transform_documents(filtered_documents)
  reordered_docs = [doc.page_content for doc in reordered_docs]
  reordered_docs=set(reordered_docs)

  #Reorded_docs가 최종 쿼리가 된다
  query_final=reordered_docs

  gen_final(query,query_final)
  print('답변 : ',gen_final(query,query_final))




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

원하는 질문을 입력하세요. 종료를 원한다면 'exit'라고 입력하세요 : 퀄컴의 핸드셋 매출이 올해 어떻게 되니


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> [INST]
        질문과 관련된 2개의 다중 검색 질문를 생성해줘.

        질문: 퀄컴의 핸드셋 매출이 올해 어떻게 되니 [/INST]
        output (주어진 질문과 유사한 질문 2개):
    
   1. 퀄컴의 핸드셋 매출은 올해 어떻게 예상되나요?
   2. 퀄컴의 핸드셋 매출이 지난해 보다 어떻게 증가할 것으로 예상되나요?</s>
<s> [INST]
        사실만을 말하는 금융 전문가로서 답해줘.

        질문: 퀄컴의 핸드셋 매출이 올해 어떻게 되니 [/INST]

    

...

답변: 최근 공시된 퀄컴의 핸드셋 매출은 112억 달러


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1033 > 1024). Running this sequence through the model will result in indexing errors


<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {'퀄컴의 QCT 매출액 중 Handsets 매출액은 2023년 2분기에 6,105백만달러 입니다.', '퀄컴의 Gross cash flow는 2020년에 5,918백만달러 입니다.\n퀄컴의 Gross cash flow는 2021년에 7,766백만달러 입니다.', '퀄컴의 매출액 총계는 2022년 3분기에 10,928백만달러 입니다.\n퀄컴의 매출액 총계는 2022년 4분기에 11,387백만달러 입니다.', '퀄컴의 영업활동에서 현금 흐름은 2020년에 5,814백만달러 입니다.\n퀄컴의 영업활동에서 현금 흐름은 2021년에 10,536백만달러 입니다.', '퀄컴의 유동자산 중 매출채권

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


퀄컴의 핸드셋 매출액은 2023년 2분기에 6,105백만달러입니다.</s>
<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {'퀄컴의 QCT 매출액 중 Handsets 매출액은 2023년 2분기에 6,105백만달러 입니다.', '퀄컴의 Gross cash flow는 2020년에 5,918백만달러 입니다.\n퀄컴의 Gross cash flow는 2021년에 7,766백만달러 입니다.', '퀄컴의 매출액 총계는 2022년 3분기에 10,928백만달러 입니다.\n퀄컴의 매출액 총계는 2022년 4분기에 11,387백만달러 입니다.', '퀄컴의 영업활동에서 현금 흐름은 2020년에 5,814백만달러 입니다.\n퀄컴의 영업활동에서 현금 흐름은